In [35]:
import os
import pandas as pd
import numpy as np
import re
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# Importing datasets

In [36]:
pwd = os.getcwd()
path_true = os.path.join(pwd, 'True.csv')
df_true = pd.read_csv(path_true)
path_fake = os.path.join(pwd, 'Fake.csv')
df_false = pd.read_csv(path_fake)

In [37]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [38]:
df_false.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


## Marking true and false news and making putting them together

In [39]:
df_true['Marker'] = 1
df_false['Marker'] = 0

In [40]:
df = df_true.append(df_false)

In [41]:
df

,title,text,subject,date,Marker
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0


# EDA

In [42]:
df.head()

,title,text,subject,date,Marker
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [43]:
df.shape

(44898, 5)

In [44]:
df.columns

Index(['title', 'text', 'subject', 'date', 'Marker'], dtype='object')

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44898 entries, 0 to 23480
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   Marker   44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.1+ MB


In [46]:
df.isnull().sum()

title      0
text       0
subject    0
date       0
Marker     0
dtype: int64

# NLP

In [47]:
input_arr = np.array(df['title'])

In [48]:
input_arr

array(['As U.S. budget fight looms, Republicans flip their fiscal script',
       'U.S. military to accept transgender recruits on Monday: Pentagon',
       "Senior U.S. Republican senator: 'Let Mr. Mueller do his job'",
       ...,
       'Sunnistan: US and Allied ‘Safe Zone’ Plan to Take Territorial Booty in Northern Syria',
       'How to Blow $700 Million: Al Jazeera America Finally Calls it Quits',
       '10 U.S. Navy Sailors Held by Iranian Military – Signs of a Neocon Political Stunt'],
      dtype=object)

In [49]:
def preprocess(title):
    subbed_array = re.sub('[^a-zA-Z]', ' ', input_arr[title])
    subbed_array = subbed_array.lower()
    subbed_array = subbed_array.split()
    s = PorterStemmer()
    subbed_array = [s.stem(word) for word in subbed_array if not word in set(stopwords.words('english'))]
    newArr = ' '.join(subbed_array)
    return subbed_array

In [50]:
df = df.drop(columns=['date','subject'], axis=1)

In [51]:
df

,title,text,Marker
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,1
...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,0


In [52]:
corpus_text = []

for i in range(0, len(df)):
    array = preprocess(i)
    corpus_text.append(array)

In [55]:
df

,title,text,Marker
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,1
...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,0


In [53]:
corpus_text

[['u', 'budget', 'fight', 'loom', 'republican', 'flip', 'fiscal', 'script'],
 ['u', 'militari', 'accept', 'transgend', 'recruit', 'monday', 'pentagon'],
 ['senior', 'u', 'republican', 'senat', 'let', 'mr', 'mueller', 'job'],
 ['fbi', 'russia', 'probe', 'help', 'australian', 'diplomat', 'tip', 'nyt'],
 ['trump', 'want', 'postal', 'servic', 'charg', 'much', 'amazon', 'shipment'],
 ['white', 'hous', 'congress', 'prepar', 'talk', 'spend', 'immigr'],
 ['trump', 'say', 'russia', 'probe', 'fair', 'timelin', 'unclear', 'nyt'],
 ['factbox', 'trump', 'twitter', 'dec', 'approv', 'rate', 'amazon'],
 ['trump', 'twitter', 'dec', 'global', 'warm'],
 ['alabama',
  'offici',
  'certifi',
  'senat',
  'elect',
  'jone',
  'today',
  'despit',
  'challeng',
  'cnn'],
 ['jone', 'certifi', 'u', 'senat', 'winner', 'despit', 'moor', 'challeng'],
 ['new',
  'york',
  'governor',
  'question',
  'constitution',
  'feder',
  'tax',
  'overhaul'],
 ['factbox',
  'trump',
  'twitter',
  'dec',
  'vaniti',
  'fair

In [54]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(corpus_text)
X = vectorizer.transform(corpus_text).toarray()
y = df.iloc[0:len(df), 2].values

AttributeError: 'list' object has no attribute 'lower'